In [11]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path
import locale

locale.setlocale(locale.LC_TIME,'pt_BR.UTF-8')

PASTA_ORIGEM = Path(r'D:\PJES_JULHO_24')
COMPETENCIA_DATA = datetime(2024, 7, 1)
NOME_PLANILHA = 'Planilha de PJES'
COLUNAS_PARA_LEITURA = "A:L"
SKIP_ROWS = 9


# Lista de cargos cota de 300
oficiais = [c.title() for c in ["Oficial da PM", "Tenente", "Capitão", "Major", 
            "Tenente Coronel", "Coronel","Delegado de Policia",
            "Perito Criminal","Medico Legista"]]

def converter_data(valor):
    return pd.to_datetime(valor, errors='coerce', dayfirst=True)
    
#Funcao para calcular verba
def calcular_verba(data_fim, competencia_mes):
    if pd.isna(data_fim):
        return ""
    if data_fim.month == competencia_mes.month and data_fim.year == competencia_mes.year:
        return 223
    elif data_fim < competencia_mes:
        return 423
    else:
        return""

def calcular_valor(row):
    try:
        cargo = str(row['CARGO']).strip().title()
        qtd_cota = float(row['QTDCOTA'])
        if cargo in oficiais:
            return qtd_cota * 300
        else:
            return qtd_cota * 200
    except Exception as e:
        print(f"Erro ao calcular valor para linha: {row}")
        print(e)
        return 0
    
def registrar_erro(mensagem, log_path="errors.log"):
    with open(log_path, 'a', encoding='utf-8') as f:
        f.write(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - {mensagem}\n")

def main():
    # lista de armazenamento dos dataframes
    log_erros = PASTA_ORIGEM / f"log_erros_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
    dfs = []
    print(f"Iniciando o processamento dos arquivos na pasta: {PASTA_ORIGEM}")

    for arquivo in PASTA_ORIGEM.glob('*.xlsx'):
            try:
                df_original = pd.read_excel(arquivo,
                                    sheet_name = NOME_PLANILHA,
                                    skiprows=SKIP_ROWS,
                                    usecols = COLUNAS_PARA_LEITURA,
                                    engine='openpyxl'
                                    )
                saida_original = PASTA_ORIGEM / f"ORIGINAL_PEJS_{arquivo.stem}.xlsx"
                df_original.to_excel(saida_original, index=False, sheet_name='DADOS BRUTOS')
                    
                print(f"✅ Arquivo '{arquivo.name}' lido com sucesso.")

                df = df_original.copy()

                df.dropna(how='all',inplace=True)
                if df.empty:
                    print(f"⚠️ O arquivo '{arquivo}' não contém dados válidos.")
                    registrar_erro(f"O arquivo '{arquivo}' não contém dados válidos.", log_erros)
                    continue

                df.columns = df.columns.str.strip().str.upper()

                #Tratamento
                df['DT TERMINO'] = df['DT TERMINO'].apply(converter_data)
                df['QTDCOTA'] = pd.to_numeric(df['QTDCOTA'], errors='coerce').fillna(0)
                df['VALOR'] = df.apply(calcular_valor, axis=1)
                df['EXERCÍCIO'] = 2024
                df['COMPETÊNCIA'] = COMPETENCIA_DATA.strftime('%B').upper()
                df['ORIGEM COTA'] = 'DECRETO'
                df['ORIGEM DA INFORMAÇÃO'] = os.path.basename(arquivo).replace('.xlsx','')
                df['OPERATIVA QUE PRESTOU SERVIÇO'] = df['OPERATIVA']
                df['LOCAL DA PRESTAÇÃO DO SERVIÇO'] = df['TITULO'].str.strip()
                df['COTA'] = df['QTDCOTA'].astype(int)
                df['VERBA'] = df['DT TERMINO'].apply(lambda x: calcular_verba(x, COMPETENCIA_DATA))
                df['CARGO'] = df['CARGO'].str.strip().str.title()
                df['SITUAÇÃO'] = 'ATIVO'
                df['MATRICULA'] = df['MATRICULA'].astype(str).str.replace(r'\.0$','',regex=True)
                df['NOME'] = df['NOME EXTENSO'].str.strip().str.title()

                # Organizar colunas 
                colunas_finais = [
                    'EXERCÍCIO', 'COMPETÊNCIA', 'ORIGEM COTA', 'ORIGEM DA INFORMAÇÃO',
                    'OPERATIVA QUE PRESTOU SERVIÇO', 'LOCAL DA PRESTAÇÃO DO SERVIÇO',
                    'MATRICULA', 'NOME', 'COTA', 'VERBA', 'CARGO', 'SITUAÇÃO','VALOR'
                ]

                
                df_modificado = df[colunas_finais].copy()
                dfs.append(df_modificado)

            
            except Exception as e:
                    msg_erro = f"Erro ao processar o arquivo '{arquivo}': {str(e)}"
                    print("⚠️", msg_erro)
                    registrar_erro(msg_erro)
    if dfs:
        data =pd.concat(dfs, ignore_index=True)
        data['COTA'] = pd.to_numeric(data['COTA'], errors='coerce').fillna(0)

        colunas_agrupamento = [
            'EXERCÍCIO', 'COMPETÊNCIA', 'ORIGEM COTA', 'ORIGEM DA INFORMAÇÃO',
            'OPERATIVA QUE PRESTOU SERVIÇO', 'LOCAL DA PRESTAÇÃO DO SERVIÇO',
            'MATRICULA', 'NOME', 'VERBA', 'CARGO', 'SITUAÇÃO'
        ]

        data_agrupado = data.groupby(colunas_agrupamento, as_index=False).agg({
            'COTA':'sum',
            'VALOR': 'sum'
        }).sort_values(by='COTA',ascending=False)

        # Criar tabela dinamica por local
        tabela_pivot = pd.pivot_table(
            data,
            index=['MATRICULA','NOME'],
            columns='LOCAL DA PRESTAÇÃO DO SERVIÇO',
            values='COTA',
            aggfunc='sum',
            fill_value=0
        ).reset_index()

        tabela_pivot.columns.name=None 
        tabela_pivot['TOTAL GERAL']=tabela_pivot.iloc[:,2:].sum(axis=1)
        tabela_pivot = tabela_pivot.sort_values(by='TOTAL GERAL',ascending= False)

        # Criar tabela dinamica por local (apenas VERBA 223)
        tabela_verba_223 = data_agrupado[data_agrupado['VERBA'] == 223]

        tabela_pivot_223 = pd.pivot_table(
            tabela_verba_223,
            index=['MATRICULA','NOME'],
            columns='LOCAL DA PRESTAÇÃO DO SERVIÇO',
            values='COTA',
            aggfunc='sum',
            fill_value=0
        ).reset_index()

        tabela_pivot_223.columns.name=None 
        tabela_pivot_223['TOTAL GERAL']=tabela_pivot_223.iloc[:,2:].sum(axis=1)
        tabela_pivot_223 = tabela_pivot_223.sort_values(by='TOTAL GERAL',ascending= False)

        
        tabela_verba_423 = data_agrupado[data_agrupado['VERBA'] == 423]

        # Tabela total por verba
        tabela_por_verba = data_agrupado.groupby(['MATRICULA','NOME','VERBA'], as_index=False).agg({
            'COTA':'sum',
            'VALOR' :'sum'
        }).sort_values(by=['MATRICULA','VERBA'],ascending=False)

        # Filtra os dados para mostrar apenas as matrículas com Total_Cotas acima de 12
        matriculas_acima_de_12 = tabela_pivot_223[tabela_pivot_223['TOTAL GERAL'] > 12]

        # Salvar em Excel
        agora = datetime.now().strftime("%Y%m%d_%H%M%S")
        nome_saida = f"TABELA_BASE_{agora}.xlsx"
        caminho_saida = PASTA_ORIGEM / nome_saida
        nome_aba_mes = COMPETENCIA_DATA.strftime('%B').upper()
            
        with pd.ExcelWriter(caminho_saida, engine='openpyxl') as writer:
            data_agrupado.to_excel(writer, index=False, sheet_name=nome_aba_mes)
            data.to_excel(writer, index=False, sheet_name='DADOS BRUTOS')
            tabela_pivot.to_excel(writer, index=False, sheet_name='RESUMO POR LOCAL')
            matriculas_acima_de_12.to_excel(writer, index=False, sheet_name='VERBA 223 > 12')
            tabela_verba_423.to_excel(writer, index=False, sheet_name='VERBA 423') 
            

        print(f"\n✅ Arquivo salvo com sucesso em:\n{caminho_saida}")

        # Excluir log se estiver vazio
        if os.path.exists(log_erros) and os.path.getsize(log_erros) == 0:
            os.remove(log_erros)

    else:
        print("Nenhum dado processado.")

if __name__ == "__main__":
    main()  

Iniciando o processamento dos arquivos na pasta: D:\PJES_JULHO_24
⚠️ Erro ao processar o arquivo 'D:\PJES_JULHO_24\07_Pagamento_PJES_Julho_24_GABSEG.xlsx': [Errno 13] Permission denied: 'D:\\PJES_JULHO_24\\ORIGINAL_PEJS_07_Pagamento_PJES_Julho_24_GABSEG.xlsx'
✅ Arquivo '07_Pagamento_PJES_Julho_24_SEMOC.xlsx' lido com sucesso.
✅ Arquivo '07_Pagamento_PJES_Julho_24_SEPDEC.xlsx' lido com sucesso.
✅ Arquivo '07_Pagamento_PJES___Julho_24_BOMBEIROS___SDS.xlsx' lido com sucesso.
✅ Arquivo '07_Pagamento_PJES___Julho_24_CEPLANC.xlsx' lido com sucesso.
✅ Arquivo '07_Pagamento_PJES___Julho_24_CORREGEDORIA.xlsx' lido com sucesso.
✅ Arquivo '07_Pagamento_PJES___Julho_24_GGCIODS.xlsx' lido com sucesso.
✅ Arquivo '07_Pagamento_PJES___Julho_24_GGPOC.xlsx' lido com sucesso.
✅ Arquivo '07_Pagamento_PJES___Julho_24_GTA.xlsx' lido com sucesso.
✅ Arquivo '07_Pagamento_PJES___Julho_24_IITB.xlsx' lido com sucesso.
✅ Arquivo '07_Pagamento_PJES___Julho_24_MPPE.xlsx' lido com sucesso.
✅ Arquivo '07_Pagamento_PJ